# Project - Data Collection Methodologies

**Postgraduate studies in Data Science**<br/>
**Student:** Nuno Henrique Pereira Martins Pinheiro<br/>

## Airports in Portugal

This project gathers data about airports in Portugal.
A generic list of them is to be gotten and data regarding their identification, location and contacts is to be persisted.
Data related with the runways of each airport and the availability of their schedules is to be exported.

### Business Requirements

1. List all airports in Portugal;
2. Fetch the airports' identification codes, location, contacts and other basic information;
3. Fetch each airports' runways information - list and identify their runways, the measures of each one and other data;
4. Check the availability status of schedules' information for each airport - the health response of different providers;
5. Export the data on airports' runways (in 3) and schedules' availability status (in 4) to two different JSON files;
6. Persist the airports' information (in 1 and 2) in a relational database - one only table.

### Technical Requirements

- **Business layer:** Python, Jupyter Notebook
- **Export information to:** JSON files
- **Persistence layer:** SQLite

- List airports in Portugal, with identification codes and basic information: `web scraping from https://en.wikipedia.org/wiki/List_of_airports_in_Portugal` (or an offline option with a copy of the webpage's content)
- Get airports' describing data from a REST endpoint: `GET https://airport-info.p.rapidapi.com/airport?icao={icao_code}`
- Get airports' runaways from a REST endpoint: `GET https://aerodatabox.p.rapidapi.com/airports/icao/{icao_code}/runways`
- Get the availability status of airport's schedules from a REST endpoint: `GET https://aerodatabox.p.rapidapi.com/health/services/airports/{icao_code}/feeds`

- Save the obtained information:
    - JSON files:
        - Airports' runways - file named *airport-runways.json*;
        - Schedules' availability - file named *airport-schedules-status.json*;
    - Database Persistence:
        - Airports' list and detailed information - table named *Airports*, with *ICAO code* as primary key. The table must be recreated, each time this notebook runs, since it is for demonstration.

### Development

In [1]:
from bs4 import BeautifulSoup
from os.path import join
import asyncio
import json
import os
import pandas as pd
import requests
import sqlite3

#### Get a collection of Portuguese Airports internationally codified, with web scraping

In [2]:
def get_html_from_url(url):
    """
    Summary: Gets HTML content from a given URL.
    Params: 'url' - The URL to get the HTML from.
    Returns: A parsed HTML.
    """
    raw_html = requests.get(url).content
    return BeautifulSoup(raw_html, 'html.parser')

def get_html_from_file(file_location):
    """
    Summary: Gets HTML content from an existing file.
    Params: 'file_location' - Location of the file to get the HTML from.
    Returns: A parsed HTML.
    """
    with open(file_location.replace('"', ''), encoding='utf8') as infile:
        return BeautifulSoup(infile.read(), 'html.parser')

def get_html(from_location, is_online):
    """
    Summary: Gets HTML content from a given location, depending on being online or offline.
    Params: 'from_location' - Location to get the HTML from. If online, it must be an URL. If offline, it must be a file location.
            'is_online' - Boolean stating if the HTML is to get from an online location. True, if online; false, if offline.
    Returns: A parsed HTML.
    """
    return get_html_from_url(from_location) if is_online else get_html_from_file(from_location)

In [3]:
# True, to fetch from online website. False to fetch from files in 'airport_website' folder
is_online = True

html_location = 'https://en.wikipedia.org/wiki/List_of_airports_in_Portugal' if is_online else join(os.getcwd(), 'airport_website\\"List of airports in Portugal - Wikipedia.html"')

parsed_html = get_html(html_location, is_online)

In [4]:
table_class = 'wikitable sortable' if is_online else 'wikitable sortable jquery-tablesorter'

airports_table = parsed_html.find_all('table', class_=table_class)

In [5]:
def build_airport_dictionary(icao, iata, city, region, description):
    """
    Summary: Builds a dictionary with the provided airport information.
    Params: 'icao' - Airport's ICAO code.
            'iata' - Airport's IATA code.
            'city' - Airport's city.
            'region' - Airport's region.
            'description' - Airport's description.
    Returns: A dictionary with the params' labels as keys to the matching input values.
    """
    return {'icao': icao, 'iata': iata, 'city': city, 'region': region, 'description': description}

def get_valid_row_attributes(table_row):
    """
    Summary: Gets the valid attributes from a given HTML table row - based on the tag "td".
    Params: 'table_row' - The table row.
    Returns: An array of contents extracted from the row.
    """
    array_to_exclude = ['\n']
    row_attributes = airport_row.find_all('td')
    row_attributes = [attribute.contents for attribute in row_attributes]
    row_attributes = [attribute for attribute in row_attributes if attribute != array_to_exclude]
    return row_attributes


# Array of dictionaries - each of them defines an airport

airports_dictionaries = []

for airport_row in airports_table[0].find_all('tr'):
    row_attributes = get_valid_row_attributes(airport_row)
    
    # Exclude airports without ICAO, but include the ones without IATA
    is_airport_without_iata = len(row_attributes) == 4 and len(row_attributes[2]) == 1
    is_airport_with_iata = len(row_attributes) == 5 and len(row_attributes[2]) == 1
    
    if is_airport_without_iata:
        icao = str(row_attributes[2][-1]).strip('\n').strip(' ')
        iata = None
        city = row_attributes[0][0].contents[0].strip('\n').strip(' ')
        region = row_attributes[1][0].contents[0].strip('\n').strip(' ')
        description = row_attributes[3][0].contents[0].strip('\n').strip(' ')
        
        airports_dictionaries.append(build_airport_dictionary(icao, iata, city, region, description))
    
    elif is_airport_with_iata:
        icao = str(row_attributes[2][-1]).strip('\n').strip(' ')
        iata = ''.join([c for c in row_attributes[3][-1] if c.isupper()])[-3:4].strip('\n').strip(' ')
        city = row_attributes[0][0].contents[0].strip('\n').strip(' ')
        region = row_attributes[1][0].contents[0].strip('\n').strip(' ')
        description = row_attributes[4][0].contents[0].strip('\n').strip(' ')
        
        airports_dictionaries.append(build_airport_dictionary(icao, iata, city, region, description))

In [6]:
# Dataframe with the Portuguese airports' basic information

airports = pd.DataFrame(airports_dictionaries)

# Drop duplicates since some airports may be registered for civil and militar uses, for example.
# However, they are the same airport, with the same ICAO
airports.drop_duplicates(subset='icao', keep='first', inplace=True)
airports = airports.reset_index(drop=True)

airports

,icao,iata,city,region,description
0,LPBJ,BYJ,Beja,Alentejo,Beja Airport
1,LPBR,BGZ,Braga,Norte,Braga Airport
2,LPBG,BGC,Bragança,Norte,Bragança Airport
3,LPCS,CAT,Cascais,Lisboa,Lisbon-Cascais Regional Airport
4,LPCH,CHV,Chaves,Norte,Chaves Airport
5,LPCO,CBP,Coimbra,Centro,Coimbra Airport
6,LPIN,None,Espinho,Norte,Espinho Airport
7,LPEV,None,Évora,Alentejo,Évora Airport
8,LPFR,FAO,Faro,Algarve,Faro Airport
9,LPVL,None,Maia,Norte,Maia Airport


#### Get detailed information for each airport, using 3 API endpoints

In [7]:
def get_http_request(url, headers, query_params=None):
    """
    Summary: Performs a GET HTTP request.
    Params: 'url' - The URL of the request.
            'headers' - The headers of the request.
            'query_params' - Query string parameters [default is None].
    Returns: The response of the HTTP request. If any unsuccessful response is returned, or if an exception occurs, a message is printed.
    """
    result = None
    
    try:
        if (query_params == None or len(query_params) == 0):
            result = requests.request('GET', url, headers=headers)
        else:
            result = requests.request('GET', url, headers=headers, params=query_params)
            
        if ((not result.ok) or result.is_redirect):
            print(f'GET response was not successful. || URL: {url} || HTTP status code: {result.status_code}')
            result = None
            
    except Exception as exc:
        print(f'The GET request raised an exception: {exc!r}')
    
    return result

def get_airport_info(icao_code):
    """
    Summary: GET request to 'https://airport-info.p.rapidapi.com/airport?icao={icao_code}'.
             Obtains basic information about a given airport, based on its ICAO code.
    Params: 'icao_code' - The ICAO code.
    Returns: HTTP response with information summary about the airport.
    """
    url = 'https://airport-info.p.rapidapi.com/airport'
    query_params = { 'icao': icao_code }
    headers = {
        'x-rapidapi-key': 'the key', # auth key is required
        'x-rapidapi-host': 'airport-info.p.rapidapi.com'
    }
    
    return get_http_request(url, headers, query_params)

def get_airport_runways(icao_code):
    """
    Summary: GET request to 'https://aerodatabox.p.rapidapi.com/airports/icao/{icao_code}/runways'.
             Based on a given airport's ICAO, it gets the airport's runways information.
    Params: 'icao_code' - The ICAO code.
    Returns: HTTP response with information summary on the airport's runways.
    """
    url = f'https://aerodatabox.p.rapidapi.com/airports/icao/{icao_code}/runways'
    headers = {
        'x-rapidapi-key': 'the key', # auth key is required
        'x-rapidapi-host': 'aerodatabox.p.rapidapi.com'
    }
    
    return get_http_request(url, headers)

def get_airport_schedules_status(icao_code):
    """
    Summary: GET request to 'https://aerodatabox.p.rapidapi.com/health/services/airports/{icao_code}/feeds'.
             Based on a given airport's ICAO, it gets availability status on the airport's schedules, in different providers.
    Params: 'icao_code' - The ICAO code.
    Returns: HTTP response with the availability status of information related with the airport's schedules.
    """
    url = f'https://aerodatabox.p.rapidapi.com/health/services/airports/{icao_code}/feeds'
    headers = {
        'x-rapidapi-key': 'the key', # auth key is required
        'x-rapidapi-host': 'aerodatabox.p.rapidapi.com'
    }
    
    return get_http_request(url, headers)

In [8]:
# Dictionaries with information related with:
# - airport descriptions, contacts, location and others
# - airport runways
# - availability status of airports' schedules

airport_info_dict = {}
airport_runways_dict = {}
airport_schedules_status_dict = {}

# Add elements to the dictionaries asynchronously to reduce operating time - the tasks can be parallelly scheduled and ran

async def add_airport_info_async(icao):
    response_info = get_airport_info(icao)
    if (response_info is not None):
        airport_info_dict[icao] = json.loads(response_info.text)
        
async def add_airport_runways_async(icao):
    response_runways = get_airport_runways(icao)
    if (response_runways is not None):
        airport_runways_dict[icao] = json.loads(response_runways.text)
        
async def add_airport_schedules_async(icao):
    response_schedules = get_airport_schedules_status(icao)
    if (response_schedules is not None):
        airport_schedules_status_dict[icao] = json.loads(response_schedules.text)

        
tasks = []

print('Start airports iterations')
for icao in airports['icao']:
    print(icao)
    tasks.append(asyncio.create_task(add_airport_info_async(icao)))
    tasks.append(asyncio.create_task(add_airport_runways_async(icao)))
    tasks.append(asyncio.create_task(add_airport_schedules_async(icao)))
    
print('Awaiting tasks')    
await asyncio.gather(*tasks)

Start airports iterations
LPBJ
LPBR
LPBG
LPCS
LPCH
LPCO
LPIN
LPEV
LPFR
LPVL
LPMI
LPMF
LPMO
LPPT
LPPM
LPPR
LPPV
LPSC
LPVR
LPVZ
LPAR
LPMR
LPMT
LPOT
LPOV
LP77
LPST
LPTN
LPCR
LPHR
LPFL
LPGR
LPPI
LPAZ
LPSJ
LPPD
LPLA
LPMA
LPPS
Awaiting tasks
GET response was not successful. || URL: https://aerodatabox.p.rapidapi.com/health/services/airports/LPCS/feeds || HTTP status code: 404
GET response was not successful. || URL: https://aerodatabox.p.rapidapi.com/health/services/airports/LPIN/feeds || HTTP status code: 404
GET response was not successful. || URL: https://aerodatabox.p.rapidapi.com/health/services/airports/LPEV/feeds || HTTP status code: 404
GET response was not successful. || URL: https://aerodatabox.p.rapidapi.com/health/services/airports/LPVL/feeds || HTTP status code: 404
GET response was not successful. || URL: https://aerodatabox.p.rapidapi.com/health/services/airports/LPMI/feeds || HTTP status code: 404
GET response was not successful. || URL: https://aerodatabox.p.rapidapi.com/hea

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

In [9]:
airport_info_dict

{'LPBJ': {'id': 1100,
  'iata': 'BYJ',
  'icao': 'LPBJ',
  'name': 'Beja Airport',
  'location': 'Beja, Portugal',
  'street_number': '',
  'street': '',
  'city': '',
  'county': '',
  'state': 'Beja',
  'country_iso': 'PT',
  'country': 'Portugal',
  'postal_code': '7780-745',
  'phone': '+351 284 001 020',
  'latitude': 38.063667,
  'longitude': -7.9393034,
  'uct': 60,
  'website': ''},
 'LPBR': {'id': 695,
  'iata': 'BGZ',
  'icao': 'LPBR',
  'name': 'Braga Airport',
  'location': 'Braga, Portugal',
  'street_number': '',
  'street': 'Rua do Carregal',
  'city': 'Palmeira',
  'county': '',
  'state': 'Braga',
  'country_iso': 'PT',
  'country': 'Portugal',
  'postal_code': '4700-688',
  'phone': '+351 253 626 530',
  'latitude': 41.586994,
  'longitude': -8.442929,
  'uct': 60,
  'website': 'https://aeroclubebraga.pt/'},
 'LPBG': {'id': 673,
  'iata': 'BGC',
  'icao': 'LPBG',
  'name': 'Bragança Airport',
  'location': 'Bragança, Portugal',
  'street_number': '',
  'street': 'Aeró

In [10]:
airport_runways_dict

{'LPBJ': [{'name': '01L',
   'trueHdg': 6.0,
   'length': {'meter': 3457.96,
    'km': 3.458,
    'mile': 2.149,
    'nm': 1.867,
    'feet': 11345.0},
   'width': {'meter': 60.05,
    'km': 0.06,
    'mile': 0.037,
    'nm': 0.032,
    'feet': 197.0},
   'isClosed': False,
   'location': {'lat': 38.063446, 'lon': -7.934465},
   'surface': 'Concrete',
   'displacedThreshold': {'meter': 0.0,
    'km': 0.0,
    'mile': 0.0,
    'nm': 0.0,
    'feet': 0.0},
   'hasLighting': True},
  {'name': '01R',
   'trueHdg': 5.9,
   'length': {'meter': 2958.08,
    'km': 2.958,
    'mile': 1.838,
    'nm': 1.597,
    'feet': 9705.0},
   'width': {'meter': 29.87,
    'km': 0.03,
    'mile': 0.019,
    'nm': 0.016,
    'feet': 98.0},
   'isClosed': False,
   'location': {'lat': 38.0654335, 'lon': -7.9317},
   'surface': 'Concrete',
   'displacedThreshold': {'meter': 0.0,
    'km': 0.0,
    'mile': 0.0,
    'nm': 0.0,
    'feet': 0.0},
   'hasLighting': True},
  {'name': '19L',
   'trueHdg': 185.9,
   '

In [11]:
airport_schedules_status_dict

{'LPBJ': {'flightSchedulesFeed': {'service': 'FlightSchedules',
   'status': 'Unavailable'},
  'liveFlightUpdatesFeed': {'service': 'FlightLiveUpdates',
   'status': 'Unavailable'},
  'adsbUpdatesFeed': {'service': 'AdsbUpdates', 'status': 'Unavailable'}},
 'LPBR': {'flightSchedulesFeed': {'service': 'FlightSchedules',
   'status': 'Unavailable'},
  'liveFlightUpdatesFeed': {'service': 'FlightLiveUpdates',
   'status': 'Unavailable'},
  'adsbUpdatesFeed': {'service': 'AdsbUpdates', 'status': 'Unavailable'}},
 'LPBG': {'flightSchedulesFeed': {'service': 'FlightSchedules',
   'status': 'Unavailable'},
  'liveFlightUpdatesFeed': {'service': 'FlightLiveUpdates',
   'status': 'Unavailable'},
  'adsbUpdatesFeed': {'service': 'AdsbUpdates', 'status': 'Unavailable'}},
 'LPCH': {'flightSchedulesFeed': {'service': 'FlightSchedules',
   'status': 'Unavailable'},
  'liveFlightUpdatesFeed': {'service': 'FlightLiveUpdates',
   'status': 'Unavailable'},
  'adsbUpdatesFeed': {'service': 'AdsbUpdates',

#### Save the airports' runways and schedules' availability status in a JSON file

In [12]:
def write_json_file(filename, data):
    """
    Summary: Writes data to a JSON file. If the file does not exist, it is created.
    Params: 'filename' - The name of the file.
            'data' - The data.
    """
    with open(filename, 'w') as json_file:
        json.dump(data, json_file)

write_json_file('airport-runways.json', airport_runways_dict)
write_json_file('airport-schedules-status.json', airport_schedules_status_dict)

#### Handle airports' information with Dataframes

In [13]:
# Merge basic airports' info from web scraping with more details from the information endpoint

non_required_feats = ['id', 'iata', 'name', 'location', 'city', 'error']
airport_new_data = pd.DataFrame(airport_info_dict).T.drop(labels=non_required_feats, axis=1)

airports_details = pd.merge(airports, airport_new_data, how='left', on='icao')
airports_details

,icao,iata,city,region,description,street_number,street,county,state,country_iso,country,postal_code,phone,latitude,longitude,uct,website
0,LPBJ,BYJ,Beja,Alentejo,Beja Airport,,,,Beja,PT,Portugal,7780-745,+351 284 001 020,38.0637,-7.9393,60,
1,LPBR,BGZ,Braga,Norte,Braga Airport,,Rua do Carregal,,Braga,PT,Portugal,4700-688,+351 253 626 530,41.587,-8.44293,60,https://aeroclubebraga.pt/
2,LPBG,BGC,Bragança,Norte,Bragança Airport,,Aeródromo de Bragança,,Bragança,PT,Portugal,5300-000,+351 273 381 175,41.8572,-6.70726,60,https://www.cm-braganca.pt/pages/125
3,LPCS,CAT,Cascais,Lisboa,Lisbon-Cascais Regional Airport,,,,Lisboa,PT,Portugal,2785,+351 21 445 7300,38.7219,-9.35361,60,http://aerodromo-cascais.pt/
4,LPCH,CHV,Chaves,Norte,Chaves Airport,S/N,Rua Dom Gualdim Pais,,Vila Real,PT,Portugal,5400-298,+351 276 321 995,41.7251,-7.46632,60,http://www.chaves.pt/pages/439
5,LPCO,CBP,Coimbra,Centro,Coimbra Airport,,Estrada Ponte,,Coimbra,PT,Portugal,3040-575,+351 239 947 235,40.1591,-8.47104,60,
6,LPIN,None,Espinho,Norte,Espinho Airport,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,LPEV,None,Évora,Alentejo,Évora Airport,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,LPFR,FAO,Faro,Algarve,Faro Airport,,,,Faro District,PT,Portugal,8001-701,+351 289 800 800,37.0176,-7.96972,60,http://www.ana.pt/pt-PT/Aeroportos/Algarve/Far...
9,LPVL,None,Maia,Norte,Maia Airport,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### Persist airports' detailed information in a newly created Airports table

In [14]:
conn = sqlite3.connect('airports.db')
cur = conn.cursor()

# Drop any previous Airports table - to allow the demonstration of the notebook - and create a new one

q = """
    DROP TABLE IF EXISTS Airports;
"""
cur.execute(q)

q = """
    CREATE TABLE Airports(
        Icao TEXT PRIMARY KEY,
        Iata TEXT,
        City TEXT,
        Region TEXT,
        Description TEXT,
        Street_Number TEXT,
        Street TEXT,
        County TEXT,
        State TEXT,
        Country_Iso TEXT,
        Country TEXT,
        Postal_Code TEXT,
        Phone TEXT,
        Latitude TEXT,
        Longitude TEXT,
        Uct TEXT,
        Website TEXT);
"""
cur.execute(q)

In [15]:
# Persist the 'airports_details' dataframe into the database - Airports table

airports_details.to_sql('Airports', con=conn, if_exists='append', index=False)

In [16]:
# Read all the data in the table Airports into a dataframe, to check the written information

q = """
    SELECT * 
    FROM Airports
"""
cur.execute(q)

sql_result = cur.fetchall()
pd.DataFrame(sql_result)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
0,LPBJ,BYJ,Beja,Alentejo,Beja Airport,,,,Beja,PT,Portugal,7780-745,+351 284 001 020,38.063667,-7.9393034,60,
1,LPBR,BGZ,Braga,Norte,Braga Airport,,Rua do Carregal,,Braga,PT,Portugal,4700-688,+351 253 626 530,41.586994,-8.442929,60,https://aeroclubebraga.pt/
2,LPBG,BGC,Bragança,Norte,Bragança Airport,,Aeródromo de Bragança,,Bragança,PT,Portugal,5300-000,+351 273 381 175,41.85719,-6.70726,60,https://www.cm-braganca.pt/pages/125
3,LPCS,CAT,Cascais,Lisboa,Lisbon-Cascais Regional Airport,,,,Lisboa,PT,Portugal,2785,+351 21 445 7300,38.721943,-9.353611,60,http://aerodromo-cascais.pt/
4,LPCH,CHV,Chaves,Norte,Chaves Airport,S/N,Rua Dom Gualdim Pais,,Vila Real,PT,Portugal,5400-298,+351 276 321 995,41.72512,-7.4663243,60,http://www.chaves.pt/pages/439
5,LPCO,CBP,Coimbra,Centro,Coimbra Airport,,Estrada Ponte,,Coimbra,PT,Portugal,3040-575,+351 239 947 235,40.15911,-8.471043,60,
6,LPIN,None,Espinho,Norte,Espinho Airport,None,None,None,None,None,None,None,None,None,None,None,None
7,LPEV,None,Évora,Alentejo,Évora Airport,None,None,None,None,None,None,None,None,None,None,None,None
8,LPFR,FAO,Faro,Algarve,Faro Airport,,,,Faro District,PT,Portugal,8001-701,+351 289 800 800,37.017597,-7.96972,60,http://www.ana.pt/pt-PT/Aeroportos/Algarve/Far...
9,LPVL,None,Maia,Norte,Maia Airport,None,None,None,None,None,None,None,None,None,None,None,None


In [17]:
conn.close()